In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import seaborn as sns
from statsmodels.api import OLS,Logit
from sklearn.metrics import roc_curve,roc_auc_score
from scipy.stats import *
from scipy.special import logit,expit

In [2]:
dataset='all_z_center_p001_20200408_500PCs'
npc=300 

# phenotypes to analyze
phe_codes=['INI21001','HC326','HC328'] 

# for regression models
covariate=['age','sex']+['PC'+str(i+1) for i in range(4)]

# load data
z=np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/train/v2/tsvd/'+dataset+'.npz', allow_pickle=True)
scores=pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/v2/'+dataset+'.sscore',
                     index_col='#IID')
phenos=pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/master.20200420.phe',
                     usecols=['IID']+phe_codes+covariate,
                     index_col='IID',
                     na_values=-9)

# population groups
train=set(pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/population-split/'+
                         'ukb24983_white_british_valid.phe').iloc[:,0].astype(float).tolist())
valid=set(pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/'+
                         'ukb24983_non_british_white.phe').iloc[:,0].astype(float).tolist())

# phenotype names
with open('/oak/stanford/groups/mrivas/users/magu/repos/rivas-lab/ukbb-tools/05_gbe/icdinfo.txt','r') as f:
    code_to_name = {line.split()[0]:line.split()[2].replace('_',' ').split('(')[0].capitalize() for line in f}

# program some special cases
code_to_name['INI21001'] = 'Body mass index'
code_to_name['HC326'] = 'Myocardial infarction'
short={'INI21001':'BMI', 'HC326':'MI', 'HC328':'Gout'}

In [3]:
width = 2
# initialize plot objects
plots = [None for _ in range(width * len(phe_codes))]
plt.close('all'); plt.clf()
fig = plt.figure(figsize=(5*width + 1, 4*len(phe_codes)), dpi=300)
grid=gs.GridSpec(4*len(phe_codes), 5*width + 1, wspace=4, hspace=2)
for i,phe_code in enumerate(phe_codes):
    i = int(i)
    for ix in range(width):
        plots[(width*i)+ix]=plt.subplot(grid[i*4:(i+1)*4, (6*ix):(6-(7*ix))])
    # scores and phenotypes
    phe_name=code_to_name[phe_code]
    df=phenos.loc[valid,[phe_code]+covariate].dropna()
    dg=phenos.loc[train,[phe_code]+covariate].dropna()
    
    n=df.shape[0]
    weights=z['V'][np.where(z['label_phe_code'] == phe_code),:].flatten()[:npc]
    df['SCORE']=zscore(scores.loc[df.index.tolist(),
                                  ['SCORE{}_SUM'.format(k+1) for k in range(npc)]].dot(weights))
    dg['SCORE']=zscore(scores.loc[dg.index.tolist(),
                                  ['SCORE{}_SUM'.format(k+1) for k in range(npc)]].dot(weights))  
    # add prs, sort by value
    prs=pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/PRS/train/v2/'+dataset[:-7]+
                      '/'+phe_code+'_PRS.profile', 
                      sep='\s+',
                      usecols=['IID','SCORESUM'], index_col='IID')
    df['PRS']=zscore(prs.loc[df.index,'SCORESUM'])
    dg['PRS']=zscore(prs.loc[dg.index,'SCORESUM'])
    df.sort_values(by='SCORE',ascending=True,inplace=True)
    dg.sort_values(by='SCORE',ascending=True,inplace=True)
    # this is to add an intercept in the regression models below
    df['1']=1
    dg['1']=1
    covariates=covariate+['1']
    # start with check for bin/qt, and plotting distributions/AUC
    joint_model={}
    if len(df[phe_code].value_counts()) == 2:
        is_bin=True
        df[phe_code]-=1 # remove plink binary format
        dg[phe_code]-=1
        # loop over dPRS and these joint models
        cov_mdl=Logit(dg[phe_code],dg[covariates]).fit(disp=0)
        cov_prs=cov_mdl.predict(df[covariates])
        for prs in ['PRS','SCORE']:
            joint_model[prs]=Logit(dg[phe_code],dg[covariates+[prs]]).fit(disp=0).predict(df[covariates+[prs]])        
        for w,(name,prs) in enumerate(zip(['PRS','dPRS','Covariate','Joint'],
                                          [df['PRS'],df['SCORE'],cov_prs,joint_model[prs]])):
            # compute stats, plot them
            fpr,tpr,_=roc_curve(df[phe_code],prs)
            roc_auc=roc_auc_score(df[phe_code],prs)
            plots[(2*i)+1].plot(fpr,tpr,label=name+' AUC: '+'{:.3f}'.format(roc_auc), 
                                color='b' if w%2 else 'grey', alpha=0.7+0.3*(w%2), linestyle='-' if w/2 else '-.')
        # add null line, axis labels
        plots[(2*i)+1].plot([0, 1],[0, 1],'k--',label='Random classification')
        plots[(2*i)+1].legend(fontsize=9)
        plots[(2*i)+1].set_xlim([0.0, 1.0])
        plots[(2*i)+1].set_ylim([0.0, 1.05])
        plots[(2*i)+1].set_ylabel('TPR', rotation=270)
        plots[(2*i)+1].set_xlabel('FPR')
    else:
        is_bin=False
        # plot correlation with dPRS and residual phenotype
        cov_mdl=OLS(dg[phe_code],dg[covariates]).fit(disp=0)
        cov_prs=cov_mdl.predict(df[covariates])
        for prs in ['PRS','SCORE']:
            joint_model[prs]=zscore(OLS(dg[phe_code],dg[covariates+[prs]]).fit(disp=0).predict(df[covariates+[prs]]))
            dprs=OLS(df[phe_code],df[covariates]).fit(disp=0).resid_pearson
            sns.regplot(x=df[prs], y=dprs, ax=plots[(2*i)+1],
                        scatter_kws={'s':2, 'alpha':0.1+0.05*(prs!='PRS')}, color='b' if prs!='PRS' else 'grey')
        plots[(2*i)+1].set_ylabel(code_to_name[phe_code]+' Residual', rotation=270)
        plots[(2*i)+1].legend([prs.replace('SCORE','dPRS')+' '+
                               '$r=$'+'{:.3f}'.format(pearsonr(dprs,df[prs])[0]) for prs in ['PRS','SCORE']])
        plots[(2*i)+1].set_xlabel('(d)PRS')
    plots[(2*i)+1].yaxis.tick_right()
    plots[(2*i)+1].yaxis.set_label_position("right")
    # now do quantile plot with modified (only if binary) phenotype
    qs=[(q+5,q) for q in np.arange(5,100,5)[::-1]] + [(5,2),(2,0)]
    print(phe_code)
    def prs_betaci((q0,q1), prs):
        we_print=(q0==2)
        q0=df[prs].quantile((100-q0)/100.0),  # pandas has 99 as the highest; we have 1 as the highest
        q1=df[prs].quantile((100-q1)/100.0)
        q40=df[prs].quantile(0.4)
        q60=df[prs].quantile(0.6)   
        iids=df.index[((q0 <= df[prs]) & (df[prs] <= q1)) | ((q40 <= df[prs]) & (df[prs] <= q60))]
        if is_bin:
            data=np.vstack((expit(cov_mdl.predict(df.loc[iids,covariates])), 
                            (q0 <= df.loc[iids,prs]) & (df.loc[iids,prs] <= q1))).T
            m=Logit(df.loc[iids,phe_code], data).fit(disp=0)
            b=np.exp(m.params[1])
            ci=np.abs(np.exp(m.conf_int().iloc[1,:].values)-b)
        else:
            data=np.vstack((cov_mdl.predict(df.loc[iids,covariates]), 
                            (q0 <= df.loc[iids,prs]) & (df.loc[iids,prs] <= q1))).T
            m=OLS(df.loc[iids,phe_code], data).fit(disp=0)
            b=m.params[1]
            ci=np.abs(m.conf_int().iloc[1,:].values-b)
        if we_print:
            print(b, ci)
        return b,ci,df.loc[(q0 <= df[prs]) & (df[prs] <= q1),phe_code].mean()
    
    for prs,name in zip(['PRS','SCORE'],['PRS','dPRS']):
        params=[prs_betaci(q, prs) for q in qs]
        plots[2*i].errorbar(x=np.arange(len(qs)), y=[p[0] for p in params], yerr=np.array([p[1] for p in params]).T,
                            fmt='o', c='b' if name=='dPRS' else 'grey', alpha=0.4+0.6*(name=='dPRS'))
        if name=='dPRS':
            p2=df.loc[(df[prs].quantile(0.4) <= df[prs]) & (df[prs] <= df[prs].quantile(0.6)), phe_code].mean()
            aax=plots[2*i].twinx()
            aax.set_yticks(plots[2*i].get_yticks())
            if is_bin:
                aax.set_yticklabels(['{:.3f}'.format(k*p2/(1-p2+(k*p2))) if k > 0 else '' 
                                     for k in plots[2*i].get_yticks()])
            else:
                aax.set_yticklabels(['{:.1f}'.format(k+p2) for k in plots[2*i].get_yticks()])
            aax.set_ylim(plots[2*i].get_ylim())
    # labels
    plots[2*i].set_ylabel(phe_name+' {}'.format('OR/prevalence' if is_bin else 'beta/mean'))
    plots[2*i].set_xticks(np.arange(len(qs)))
    plots[2*i].legend(['PRS','dPRS','Mean'], loc=2, fontsize=9)
    plots[2*i].plot(plots[2*i].get_xlim(),[is_bin,is_bin],'k--',alpha=0.2)
    # letters
    plots[2*i].text(-0.1, 1.05, chr((i%len(phe_codes))+65), 
                    fontsize=16, transform=plots[2*i].transAxes)
    plots[(2*i)+1].text(-0.1, 1.05, chr((i%len(phe_codes))+65+len(phe_codes)), 
                        fontsize=16, transform=plots[(2*i)+1].transAxes)
    # x-axis labels at the very end
    if i==len(phe_codes)-1:
        plots[(2*i)].set_xlabel('(d)PRS quantile (non-British Whites)')
        plots[2*i].set_xticklabels(list(map(str,qs)), rotation=90)
    else:
        plots[(2*i)].set_xlabel('')
        plots[2*i].set_xticklabels(['' for _ in plots[2*i].get_xticklabels()])

plt.show()
fig.savefig("Figure-S2.pdf", bbox_inches='tight')

SyntaxError: invalid syntax (<ipython-input-3-f2ee370622f7>, line 79)

In [ ]:
len(train),len(valid)